In [3]:
import pandas as pd
import numpy as np
col_headers = ['TrackID', 'Predictor', 'rawPrediction', 'probability']
clean_data = pd.read_csv('./unsupervised_dtModel.csv') 
clean_data.columns = col_headers
print(clean_data.head())

new_predictors = []
new_predictors_index = 0

for k,g in clean_data.groupby(np.arange(len(clean_data))//6):
    batched = g
    num_ones = 0
    num_zeros = 0
    index_count = 0
    index_one = []
    index_zero = []
    
    batch_probability = batched['probability']
    batch_predictor = batched['Predictor']
    batch_rawPred = batched['rawPrediction']
    
    for i in batch_predictor:
        new_predictors.append(i)
        new_predictors_index = new_predictors_index + 1
        if (i == 0):
            num_zeros = num_zeros + 1
            index_zero.append(index_count)
        if (i == 1):
            num_ones = num_ones + 1
            index_one.append(index_count)
        index_count = index_count + 1
    #print('num_zeros: ', num_zeros)
    
    index_count = 0
    index_rawPred_zero = []
    index_rawPred_one = []
    if (num_zeros > 3):
        for i in batch_rawPred:
            preprocess = i
            preprocess = preprocess[1:]
            preprocess = preprocess[:len(preprocess)-1]
            preprocess = preprocess.split(',')
            preprocess = preprocess[0]
            preprocess = float(preprocess)
            if (index_count in index_zero):
                index_rawPred_zero.append(preprocess)
            index_count = index_count + 1
            
        index_rawPred_zero.sort()
        index_rawPred_zero_toremove = []
        zeros_to_correct = num_zeros - 3
        for i in range(0, len(index_rawPred_zero)):
            if (zeros_to_correct > 0):
                index_rawPred_zero_toremove.append(index_rawPred_zero[i])
            zeros_to_correct = zeros_to_correct - 1
        
        index_count = 0
        index_to_switch = []
        for i in batch_rawPred:
            preprocess = i
            preprocess = preprocess[1:]
            preprocess = preprocess[:len(preprocess)-1]
            preprocess = preprocess.split(',')
            preprocess = preprocess[0]
            preprocess = float(preprocess)
            if preprocess in index_rawPred_zero_toremove:
                index_to_switch.append(index_count)
            index_count = index_count + 1
            
        index_count = 0
        for i in range(0, 6):
            if index_count in index_to_switch:
                new_predictors[k*6+index_count] = 1
            index_count = index_count + 1
    
    index_count = 0
    if (num_ones > 3):
        for i in batch_rawPred:
            preprocess = i
            preprocess = preprocess[1:]
            preprocess = preprocess[:len(preprocess)-1]
            preprocess = preprocess.split(',')
            preprocess = preprocess[0]
            preprocess = float(preprocess)
            preprocess = abs(preprocess)
            if (index_count in index_one):
                index_rawPred_one.append(preprocess)
            index_count = index_count + 1
            
        index_rawPred_one.sort()
        index_rawPred_one_toremove = []
        ones_to_correct = num_ones - 3
        for i in range(0, len(index_rawPred_one)):
            if (ones_to_correct > 0):
                index_rawPred_one_toremove.append(index_rawPred_one[i])
            ones_to_correct = ones_to_correct - 1
        
        index_count = 0
        index_to_switch = []
        for i in batch_rawPred:
            preprocess = i
            preprocess = preprocess[1:]
            preprocess = preprocess[:len(preprocess)-1]
            preprocess = preprocess.split(',')
            preprocess = preprocess[0]
            preprocess = float(preprocess)
            preprocess = abs(preprocess)
            if preprocess in index_rawPred_one_toremove:
                index_to_switch.append(index_count)
            index_count = index_count + 1
            
        index_count = 0
        for i in range(0, 6):
            if index_count in index_to_switch:
                new_predictors[k*6+index_count] = 0
            index_count = index_count + 1

clean_data['Predictor'] = new_predictors
print(clean_data.head())

unsuper_col = ['TrackID', 'Predictor']
cleaned_data = clean_data[['TrackID', 'Predictor']].copy()
cleaned_data.to_csv('./dt_cleaned_pyspark_1211.csv', index=False, header=unsuper_col)

         TrackID  Predictor rawPrediction  \
0   199810_74139        0.0   [108.0,4.0]   
1  199810_242681        0.0   [108.0,4.0]   
2    199810_9903        0.0    [12.0,2.0]   
3  199810_208019        1.0     [0.0,2.0]   
4   199810_18515        1.0    [0.0,36.0]   

                                probability  
0  [0.9642857142857143,0.03571428571428571]  
1  [0.9642857142857143,0.03571428571428571]  
2  [0.8571428571428571,0.14285714285714285]  
3                                 [0.0,1.0]  
4                                 [0.0,1.0]  
         TrackID  Predictor rawPrediction  \
0   199810_74139        0.0   [108.0,4.0]   
1  199810_242681        0.0   [108.0,4.0]   
2    199810_9903        0.0    [12.0,2.0]   
3  199810_208019        1.0     [0.0,2.0]   
4   199810_18515        1.0    [0.0,36.0]   

                                probability  
0  [0.9642857142857143,0.03571428571428571]  
1  [0.9642857142857143,0.03571428571428571]  
2  [0.8571428571428571,0.14285714285714285] 